### Step 4 - Generate Results

This script has gone through a lot of development, and is now in two version - with the 'operational' one being the automated version also labelled as Step 4 in this folder. 

This is really where most of the complexity lies in the Yemen analysis, as we have been asked to do cuts and analyses for the WHO and UNICEF teams on the ground in Yemen that no other team has (as of yet!) requested from the GOST team. 

This is a good script for learning what happens in the even more complicated Automated version. 

Import the usual suspects

In [2]:
import pandas as pd
import os, sys
sys.path.append(r'C:\Users\charl\Documents\GitHub\GOST_PublicGoods\GOSTNets\GOSTNets')
sys.path.append(r'C:\Users\charl\Documents\GitHub\GOST')
import GOSTnet as gn
import importlib
import geopandas as gpd
import rasterio as rt
from rasterio import features
from shapely.wkt import loads
import numpy as np
import networkx as nx
from shapely.geometry import box, Point, Polygon

peartree version: 0.6.1 
networkx version: 2.3 
matplotlib version: 3.0.3 
osmnx version: 0.9 


### Settings

Here, we build our scenario for this run through of Generate Results (which must be run multiple times for multiple scenarios). 

It is this bit which is effectively automated in the 'automated version' of the script. Here, we set the 'scenario variables':
- whether or not to model access to a motor vehicle for driving along roads; 
- whether to incorporate a version of the graph with adjustments made for conflict (i.e. road closures, conflicts); 
- the year of the analysis;
- whether we are looking at access to hosptials, Primary HealthCare facilities (PHCs) or both, 
- the range of services which we are analyzing access to (a subset of the facilities)

In [2]:
walking = 0 # set to 1 for walking, 0 for driving.

conflict = 1 # set to 1 to prevent people from crossing warfronts, and to incorporate road closures per UN logistics cluster data

facility_type = 'ALL'   # Options: 'HOS' or 'PHC' or 'ALL'

year = 2018   # default = 2018; can be 2016 if 2016 origin layer prepared

service_index = 8 # Set to 0 for all services / access to hospitals. A choice from the next list.

services = ['ALL',
            'Antenatal',
            'BEmONC',
            'CEmONC',
            'Under_5',
            'Emergency_Surgery',
            'Immunizations',
            'Malnutrition',
            'Int_Outreach']

zonal_stats = 1 # set to 1 to produce summary zonal stats layer

### Import All-Destination OD

The OD will have on one axis all of the uniquely snapped-to nodes for the origin points, and on the other, all of the uniquely snapped-to nodes amongst the destinations. 

In [3]:
basepth = r'C:\Users\charl\Documents\GOST\Yemen'

# path to the graphtool folder for the files that generated the OD matrix
pth = os.path.join(basepth, 'graphtool')

# path for utility files, e.g. admin boundaries, warfront file
pth = os.path.join(basepth, 'util_files')

# path to the SRTM tiles used for matching on node elevation
pth = os.path.join(basepth, 'SRTM')

In this block, we translate some of the settings elected above into file name suffixes to help us tell different outputs apart.

In [1]:
# in the event walking is on, we want the walk graph, not the normal driving graph. 
if walking == 1:
    type_tag = 'walking'
    net_name = r'walk_graph.pickle'
else:
    type_tag = 'driving'
    # this network has both conflict and non-conflict drive times on its edges. 
    net_name = r'G_salty_time_conflict_adj.pickle'

# if we are looking at conflict, at the ConflictAdj suffix to output files
if conflict == 1: 
    conflict_tag = 'ConflictAdj'
else:
    conflict_tag = 'NoConflict'

# our path to our OD matrix should be the graphtool folder (most of the time!). Adjust if necessary
OD_pth = pth
OD_name = r'output_Jan24th_%s.csv' % type_tag

# same is also true of our network path - adjust if necessary
net_pth = pth

# Define our CRS for the analysis - WGS84 and projected. 
WGS = {'init':'epsg:4326'}
measure_crs = {'init':'epsg:32638'}

# Here we build the whole filename based on our choice of settings, and print those beneath
subset = r'%s_24th_HERAMS_%s_%s_%s_%s' % (type_tag, facility_type, services[service_index], conflict_tag, year)
print("Output files will have name: ", subset)
print("network: ",net_name)
print("OD Matrix: ",OD_name)
print("Conflict setting: ",conflict_tag)

# set the speed at which people are assumed to move over open ground with no paths
# e.g. when walking from desitnation node to actual destination. KEY ASSUMPTION!!
offroad_speed = 4

NameError: name 'walking' is not defined

### Read in OD Matrix
Here, we read in our OD matrix, and do a bit of housekeeping on it. 

In [5]:
OD = pd.read_csv(os.path.join(OD_pth, OD_name))
OD = OD.rename(columns = {'Unnamed: 0':'O_ID'})
OD = OD.set_index('O_ID')
OD = OD.replace([np.inf, -np.inf], np.nan)
OD_original = OD.copy()

### Subset to Accepted Nodes
In this block, we do simila operations to the HeRAMS file, including:
- removing facilities which aren't of the selected type
- removing non-operational facilities
- If selected, dropping facilities which don't offer the required services

In [6]:
# initial read in
acceptable_df = pd.read_csv(os.path.join(OD_pth, 'HeRAMS 2018 April_snapped.csv'))

# Adjust for facility type - drop facilities as necessary. 1 = hospital, 2/3 = PHC
if facility_type == 'HOS':
    acceptable_df = acceptable_df.loc[acceptable_df['Health Facility Type Coded'].isin(['1',1])]
elif facility_type == 'PHC':
    acceptable_df = acceptable_df.loc[acceptable_df['Health Facility Type Coded'].isin([2,'2',3,'3'])]
elif facility_type == 'ALL':
    pass
else:
    raise ValueError('unacceptable facility_type entry!')

# Adjust for functionality in a given year. Functioning facilities given by a 1 or a 2.
acceptable_df = acceptable_df.loc[acceptable_df['Functioning %s' % year].isin(['1','2',1,2])]

# Adjust for availability of service.
# first we build a dictionary of all the columns we care about with a standardized KEY (note: not value!)
# we use the keys themselves to then subset the Pandas DF
SERVICE_DICT = {'Antenatal_2018':'ANC 2018',
               'Antenatal_2016':'Antenatal Care (P422) 2016',
               'BEmONC_2018':'Basic emergency obstetric care 2018',
               'BEmONC_2016':'Basic Emergency Obsteteric Care (P424) 2016',
               'CEmONC_2018':'Comprehensive emergency obstetric care 2018',
               'CEmONC_2016':'Comprehensive Emergency Obstetric Care (S424) 2016',
               'Under_5_2018':'Under 5 clinics 2018',
               'Under_5_2016':'Under-5 clinic services (P23) 2016',
               'Emergency_Surgery_2018':'Emergency and elective surgery 2018',
               'Emergency_Surgery_2016':'Emergency and Elective Surgery (S14) 2016',
               'Immunizations_2018':'EPI 2018',
               'Immunizations_2016':'EPI (P21a) 2016',
               'Malnutrition_2018':'Malnutrition services 2018',
               'Malnutrition_2016':'Malnutrition services (P25) 2016',
               'Int_Outreach_2018':'Integrated outreach (IMCI+EPI+ANC+Nutrition_Services) 2018',
               'Int_Outreach_2016':'Integrated Outreach (P22) 2016'}

if service_index == 0:
    pass
else:
    # take this line slowly...
    acceptable_df = acceptable_df.loc[acceptable_df[SERVICE_DICT['%s_%s' % (services[service_index],year)]].isin(['1',1])]

# print out the length of the acceptable_df - which is, at this point, the list of valid destinations for this analysis. 
len(acceptable_df)

2373

### OD-Matrix slicing for valid destinations

In this section, we pick out the snapped-to nodes for the remaining valid destinations, and then slice the larger OD matrix accordingly.

In [7]:
# load the geometry column, make it a GeoDataFrame
acceptable_df['geometry'] = acceptable_df['geometry'].apply(loads)
acceptable_gdf = gpd.GeoDataFrame(acceptable_df, geometry = 'geometry', crs = {'init':'epsg:4326'})

# convert types of nearest node (currently stored as int) into string, find unique set. 
accepted_facilities = list(set(list(acceptable_df.NN)))
accepted_facilities_str = [str(i) for i in accepted_facilities]

# keep ONLY the columns in the OD-Matrix relating to nodes snapped to by these destinations. 
# Massive reduction in size of the OD-matrix in most cases
OD = OD_original[accepted_facilities_str]

# Send to file the destination .csv - used for generating outputs as the facility locations in this case
acceptable_df.to_csv(os.path.join(basepth,'output_layers','Round 3','%s.csv' % subset))

# print out dimensions to make sure we are on track. 
print(OD_original.shape)
print(OD.shape)

(36624, 3824)
(36624, 2182)


### Define function to add elevation to a point GeoDataFrame

This function takes the work in Step 3.a, and functionalizes it - allowing us to add an elevation field to a point GeoDataFrame, assuming we have a path to all the SRTM tiles and we have denoted x and y Lat / Long columns in WGS84. See Step 3.a for a detailed walkthrough of this function's methodology.

In [8]:
def add_elevation(df, x, y, srtm_pth):
    
    # walk all tiles, find path
    tiles = []
    for root, folder, files in os.walk(os.path.join(srtm_pth,'high_res')):
        for f in files:
            if f[-3:] == 'hgt':
                tiles.append(f[:-4])

    # load dictionary of tiles
    arrs = {}
    for t in tiles:
        arrs[t] = rt.open(srtm_pth+r'\high_res\{}.hgt\{}.hgt'.format(t, t), 'r')

    # assign a code
    uniques = []
    df['code'] = 'placeholder'
    def tile_code(z):
        E = str(z[x])[:2]
        N = str(z[y])[:2]
        return 'N{}E0{}'.format(N, E)
    df['code'] = df.apply(lambda z: tile_code(z), axis = 1)
    unique_codes = list(set(df['code'].unique()))
    
    z = {}
    
    # Match on High Precision Elevation
    property_name = 'elevation'
    for code in unique_codes:
        
        df2 = df.copy()
        df2 = df2.loc[df2['code'] == code]
        dataset = arrs[code]
        b = dataset.bounds
        datasetBoundary = box(b[0], b[1], b[2], b[3])
        selKeys = []
        selPts = []
        for index, row in df2.iterrows():
            if Point(row[x], row[y]).intersects(datasetBoundary):
                selPts.append((row[x],row[y]))
                selKeys.append(index)
        raster_values = list(dataset.sample(selPts))
        raster_values = [x[0] for x in raster_values]

        # generate new dictionary of {node ID: raster values}
        z.update(zip(selKeys, raster_values))
        
    elev_df = pd.DataFrame.from_dict(z, orient='index')
    elev_df.columns = ['elevation']
    
    # match on low-precision elevation
    missing = elev_df.copy()
    missing = missing.loc[missing.elevation < 0]
    if len(missing) > 0:
        missing_df = df.copy()
        missing_df = missing_df.loc[missing.index]
        low_res_tifpath = os.path.join(srtm_pth, 'clipped', 'clipped_e20N40.tif')
        dataset = rt.open(low_res_tifpath, 'r')
        b = dataset.bounds
        datasetBoundary = box(b[0], b[1], b[2], b[3])
        selKeys = []
        selPts = []
        for index, row in missing_df.iterrows():
            if Point(row[x], row[y]).intersects(datasetBoundary):
                selPts.append((row[x],row[y]))
                selKeys.append(index)
        raster_values = list(dataset.sample(selPts))
        raster_values = [x[0] for x in raster_values]
        z.update(zip(selKeys, raster_values))

        elev_df = pd.DataFrame.from_dict(z, orient='index')
        elev_df.columns = ['elevation']
    df['point_elev'] = elev_df['elevation']
    df = df.drop('code', axis = 1)
    return df

### Define function to convert distances to walk times

Once again, the Tobler's hiking function reappears from Step 3.a to help us generate elevation-adjusted walk times for our network. This verion is modified to return a walk time for the distances between:
- an origin / destination coordinate pair (the raw lat/long in the WHO's HeRAMS file or an origin centroid in WorldPop), and
- the nearest node on the network. 
This explains why the 'dist' argument is set by default to 'NN_dist' - or the distance to the nearest node for each point in the frame.

In [9]:
def generate_walktimes(df, start = 'point_elev', end = 'node_elev', dist = 'NN_dist', max_walkspeed = 6, min_speed = 0.1):
    
    def speed(incline_ratio, max_speed):
        walkspeed = max_speed * np.exp(-3.5 * abs(incline_ratio + 0.05)) 
        return walkspeed

    speeds = {}
    times = {}

    for index, data in df.iterrows():
        if data[dist] > 0:
            delta_elevation = data[end] - data[start]
            incline_ratio = delta_elevation / data[dist]
            speed_kmph = speed(incline_ratio = incline_ratio, max_speed = max_walkspeed)
            speed_kmph = max(speed_kmph, min_speed)
            speeds[index] = (speed_kmph)
            times[index] = (data[dist] / 1000 * 3600 / speed_kmph)

    speed_df = pd.DataFrame.from_dict(speeds, orient = 'index')
    time_df = pd.DataFrame.from_dict(times, orient = 'index')

    df['walkspeed'] = speed_df[0]
    df['walk_time'] = time_df[0]
    
    return df

### Add elevation for destination nodes

What it says on the tin - adding an elevation column for the dest_df, the destinations GeoDataFrame. Note we add the elevation for the _points themselves_, NOT their nearest nodes - which we will do soon.

We sneakily also set the index to 'Nearest Node' (which will always here be referred to as 'NN').

In [10]:
dest_df = acceptable_df[['NN','NN_dist','Latitude','Longitude']]
dest_df = add_elevation(dest_df, 'Longitude','Latitude', srtm_pth).set_index('NN')

C:\Users\charl\Anaconda3\envs\Cfox2\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\charl\Anaconda3\envs\Cfox2\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\charl\Anaconda3\envs\Cfox2\lib\site-packages\ipykernel_launcher.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas

### Add elevation from graph nodes

It will come in useful to also have a dataframe of our nodes with their elevation matched on. We generate this here using tried and tested methods.

In [11]:
G = nx.read_gpickle(os.path.join(OD_pth, net_name))
G_node_df = gn.node_gdf_from_graph(G)
G_node_df = add_elevation(G_node_df, 'x', 'y', srtm_pth)
match_node_elevs = G_node_df[['node_ID','point_elev']].set_index('node_ID')
match_node_elevs.loc[match_node_elevs.point_elev < 0] = 0

### Match on node elevations for dest_df nearest nodes; calculate travel times to nearest node

Here, having earlier set the dest_df index to nearest node, and having generate above a reference frame for all node elevations, we match on the elevation of each destination's _nearest node_. This will allow us to calculate walk times to the network with accuracy for each destination.

In [12]:
# match on NN elevation (not actual destination elevation!)
dest_df['node_elev'] = match_node_elevs['point_elev']

# with all our constituent fields generated, now we can generate our network-to-actual-destination walktimes.
dest_df = generate_walktimes(dest_df, start = 'node_elev', end = 'point_elev', dist = 'NN_dist', max_walkspeed = offroad_speed)

# we sort, for fun, mainly. 
dest_df = dest_df.sort_values(by = 'walk_time', ascending = False)

### Add Walk Time to all travel times in OD matrix

We have now generated one component of three of a given travel time between an origin and a destination. 

Due to the organization of the OD matrix (we only have the unique snapped-to origins nodes, NOT all 560,000 origin nodes in there...) it becomes easier to adjust every journey time to add on required walk time at the end from network to destination. 

We do that here. 

In [13]:
# subset the destination DataFrame into one that is just equal to the walk-time from network. index = NN. 
dest_df = dest_df[['walk_time']]

# convert to type that will work with the OD matrix
dest_df.index = dest_df.index.map(str)

# flip the OD-matrix! Now, origins are the columns, not the rows. 
d_f = OD.transpose()

# for each origin node column, 
for i in d_f.columns:
    
    # match on this column to the destination DataFrame (bear with me, I know this is funky)
    dest_df[i] = d_f[i]

# now, we add the walk time to each and every value, laterally:
for i in dest_df.columns:
    if i == 'walk_time':
        pass
    # executed here
    else:
        dest_df[i] = dest_df[i] + dest_df['walk_time']

# before dropping the walk_time - it has been added everywhere, afterall. 
dest_df = dest_df.drop('walk_time', axis = 1)

# then we flip our OD-matrix back the right way up, with rows once again equal to origin nodes, and columns = destination nodes. 
dest_df = dest_df.transpose()

This is the hardest bit to follow in this script. It has to be done this way with our limited computing resources (and is actually fairly efficient). Nonetheless, it isn't pretty. 

### Import Shapefile Describing Regions of Control

In this block, we import one of two shapefiles. If conflict is OFF, we have the NoConflict.shp - which is a single shape, with no warfronts in it. Otherwise, w pick up the merged_dists.shp, generated in the Prep A workbook. 

If you intend to do a scenario involving conflict, go and run that first and come back when merged_dists.shp exists. 

In [14]:
# choose relevant file
if conflict == 1:
    conflict_file = r'merged_dists.shp'
elif conflict == 0:
    conflict_file = r'NoConflict.shp'
    
# read in relevant file
merged_dists = gpd.read_file(os.path.join(util_path, conflict_file))

# project if necessary
if merged_dists.crs != {'init':'epsg:4326'}:
    merged_dists = merged_dists.to_crs({'init':'epsg:4326'})
    
# only keep Polygons (nothing else should be in there anyway)
merged_dists = merged_dists.loc[merged_dists.geometry.type == 'Polygon']

### Factor in lines of Control - Import Areas of Control Shapefile

This function helps us intersect GeoDataFrames and the warfronts file. 

It returns a dictionary where:
- the index is the polygon index in a GeoDataFrame containing polygons, 
- the value is a list of objects which fall within that polygon

It does this very quickly by leveraging spatial indices to speed up the process. Further, we cut up polygons which are too big into smaller blocks and iterate through those to further turbocharge the intersection. 

In [15]:
# Intersect points with merged districts shapefile, identify relationship
def AggressiveSpatialIntersect(points, polygons):
    # points must be a GeoDataFrame containing points
    # polygons must be a GeoDataFrame containing polygons
    
    import osmnx as ox
    
    # make a spatial index of the points to be intersected
    spatial_index = points.sindex
    
    # set up the dictionary to be returned
    container = {}
    cut_geoms = []
    
    # iterate through each polygon
    for index, row in polygons.iterrows():
        
        # pick out the shapely object
        polygon = row.geometry
        
        # the polygon is big, 
        if polygon.area > 0.5:
            
            # cut the geometry into quadrants of width 0.5 arc-seconds
            geometry_cut = ox.quadrat_cut_geometry(polygon, quadrat_width=0.5)
            
            # add this to a list of cut geometries
            cut_geoms.append(geometry_cut)
            
            # notify that we are taking some scissors to this polygon in particular
            print('cutting geometry %s into %s pieces' % (index, len(geometry_cut)))
            index_list = []
            
            # now go through these geometry pieces, and perform the spatial intersect
            for P in geometry_cut:
                
                possible_matches_index = list(spatial_index.intersection(P.bounds))
                possible_matches = points.iloc[possible_matches_index]
                precise_matches = possible_matches[possible_matches.intersects(P)]
                if len(precise_matches) > 0:
                    index_list.append(precise_matches.index)
                flat_list = [item for sublist in index_list for item in sublist]
                container[index] = list(set(flat_list))
                
        # if it is a small polygon, just go for the intersection anyway (via spatial index)
        else:
            possible_matches_index = list(spatial_index.intersection(polygon.bounds))
            possible_matches = points.iloc[possible_matches_index]
            precise_matches = possible_matches[possible_matches.intersects(polygon)]
            if len(precise_matches) > 0:
                container[index] = list(precise_matches.index)
                
    # return the dictionary of which points lie in which polygons.
    return container

Here, we deploy this function to identify which graph nodes lie in which areas of control. Possible_snap_nodes now is set to the resultant dictionary, of format: {polygon : [contained nodes list]}

In [17]:
graph_node_gdf = gn.node_gdf_from_graph(G)
gdf = graph_node_gdf.copy()
gdf = gdf.set_index('node_ID')
possible_snap_nodes = AggressiveSpatialIntersect(graph_node_gdf, merged_dists)
print('**bag of possible node snapping locations has been successfully generated**')

cutting geometry 56 into 121 pieces
cutting geometry 59 into 47 pieces
cutting geometry 78 into 236 pieces
cutting geometry 79 into 16 pieces
**bag of possible node snapping locations has been successfully generated**


### Load Origins Grid

This is the first time we pick up and work with the large origin GeoDataFrame. We load the correct one in for the year we are working with, and perform some housekeeping.

In [18]:
# Match on network time from origin node (time travelling along network + walking to destination)
if year == 2018:
    year_raster = 2018
elif year == 2016:
    year_raster = 2015
grid_name = r'origins_1km_%s_snapped.csv' % year_raster
grid = pd.read_csv(os.path.join(OD_pth, grid_name))
grid = grid.rename({'Unnamed: 0':'PointID'}, axis = 1)
grid['geometry'] = grid['geometry'].apply(loads)
grid_gdf = gpd.GeoDataFrame(grid, crs = WGS, geometry = 'geometry')
grid_gdf = grid_gdf.set_index('PointID')

### Adjust Nearest Node snapping for War

This gets kinda interesting. The first thing we do is generate the reference dictionary for the origin layer, of which origin points lie within which polygon of homogenous control (Remember if conflict is  turned off, then all points lie within one single homogenous polygon - Yemen's territorial boundaries (minue Socotra...I digress)).

Thereafter, we remake the origin file with the polygon reference for each point in the final line, where we concat 'bundle' - the slices of the original file that lie inside each polygon.

In [19]:
origin_container = AggressiveSpatialIntersect(grid_gdf, merged_dists)
print('bag of possible origins locations has been successfully generated')

bundle = []
for key in origin_container.keys():
    origins = origin_container[key]
    possible_nodes = graph_node_gdf.loc[possible_snap_nodes[key]]
    origin_subset = grid_gdf.loc[origins]
    origin_subset_snapped = gn.pandana_snap_points(origin_subset, 
                                possible_nodes, 
                                source_crs = 'epsg:4326', 
                                target_crs = 'epsg:32638', 
                                add_dist_to_node_col = True)
    bundle.append(origin_subset_snapped)

grid_gdf_adjusted = pd.concat(bundle)

cutting geometry 56 into 121 pieces
cutting geometry 59 into 47 pieces
cutting geometry 78 into 236 pieces
cutting geometry 79 into 16 pieces
bag of possible origins locations has been successfully generated


C:\Users\charl\Documents\GitHub\GOST_PublicGoods\GOSTNets\GOSTNets\GOSTnet.py:1679: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  G_tree = spatial.KDTree(target_gdf[['x','y']].as_matrix())
C:\Users\charl\Documents\GitHub\GOST_PublicGoods\GOSTNets\GOSTNets\GOSTnet.py:1681: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  distances, indices = G_tree.query(source_gdf[['x','y']].as_matrix())


We set the grid_gdf back to this conflict adjusted file - so we carry forward the changes we just made. 

(for those working through the script, might be worth comparing grid_gdf_adjusted and grid_gdf to see the change this block makes)

In [20]:
grid_gdf = grid_gdf_adjusted

### Adjust acceptable destinations for each node for the war

We repeat the process in the last cell, but this time for graph nodes snapped-to as desintations, and those snapped-to as origins

In [21]:
#### for Destination nodes (dest_Df.columns)
# take the nodes GDF
gdf = graph_node_gdf.copy()

# housekeep for ID datatype
gdf['node_ID'] = gdf['node_ID'].astype('str')

# take only the nodes that also appear in the dest_df columns (i.e. the OD matrix). Cols = Dests
gdf = gdf.loc[gdf.node_ID.isin(list(dest_df.columns))]

# set the index as the node ID
gdf = gdf.set_index('node_ID')

# generate our reference dictionary for snapped-to destination nodes
dest_container = AggressiveSpatialIntersect(gdf, merged_dists)

# repeat process for Origin nodes (dest_Df.index)
gdf = graph_node_gdf.copy()

# remember, the index of dest_df (the OD matrix) is the nodes snapped-to by origin points
gdf = gdf.loc[gdf.node_ID.isin(list(dest_df.index))]
gdf = gdf.set_index('node_ID')

# generate our reference dictionary
origin_snap_container = AggressiveSpatialIntersect(gdf, merged_dists)

cutting geometry 56 into 121 pieces
cutting geometry 59 into 47 pieces
cutting geometry 78 into 236 pieces
cutting geometry 79 into 16 pieces
cutting geometry 56 into 121 pieces
cutting geometry 59 into 47 pieces
cutting geometry 78 into 236 pieces
cutting geometry 79 into 16 pieces


### Working out the Min-time for each polygon

This is a fairly crucial block. Here, we iterate through the polygons, take only the valid origins and destinations in that polygon, and work out the minimum time, for each selected origin, to a valid destination. This is how we prevent people crossing borders - we effectively run N small-universe analyses, where the rest of the country outside the polygon doesn't exist! Remember this is on-network travel only.

In [ ]:
bundle = []

# for each polygon
for key in origin_snap_container.keys():
    
    # select the origins which exist in this polygon
    origins = origin_snap_container[key]
    
    # select the destinations which exist in this polygon
    destinations = dest_container[key]
    
    # subset the OD-matrix to include just these origins and destinations
    Q = dest_df[destinations].loc[origins]
    
    # make a new column, min-time - for each origin, its closest destination
    Q['min_time'] = Q.min(axis = 1)
    
    # subset to just this column
    Q2 = Q[['min_time']]
    
    # append to bundle
    bundle.append(Q2)

# concatenate results    
Q3 = pd.concat(bundle)

# add to the OD matrix
dest_df['min_time'] = Q3['min_time']

### Return to Normal Process

In [22]:
grid_gdf = grid_gdf.rename(columns = {'NN':'O_ID'})

### Merge on min Time

Here, we add the min-time on the network for each origin node to each origin point. This is a large 1-to-many join for each snapped-to origin node (c. 36k unique origin nodes on the network being joined over to c. 560k origin points, many of which have duplicate closest nodes, as you would expect). 

In [23]:
grid_gdf = grid_gdf.reset_index()
grid_gdf = grid_gdf.set_index(grid_gdf['O_ID'])
grid_gdf['on_network_time'] = dest_df['min_time']
grid_gdf = grid_gdf.set_index('PointID')

### Add origin node distance to network - walking time

We now have 2/3 of the puzzle done for each journey - the walk from the network to the destination (baked in to all OD matrix values now) and the on-network minimum time to a valid destination (in the same polygon as the origin point!). Now, we go ahead and add the walk time from the specific origin point to the network. 

In [24]:
grid = grid_gdf
grid = add_elevation(grid, 'Longitude','Latitude', srtm_pth)
grid = grid.reset_index()
grid = grid.set_index('O_ID')
grid['node_elev'] = match_node_elevs['point_elev']
grid = grid.set_index('PointID')
grid = generate_walktimes(grid, start = 'point_elev', end = 'node_elev', dist = 'NN_dist', max_walkspeed = offroad_speed)
grid = grid.rename({'node_elev':'nr_node_on_net_elev', 
                    'walkspeed':'walkspeed_to_net', 
                    'walk_time':'walk_time_to_net',
                   'NN_dist':'NN_dist_to_net'}, axis = 1)

# thus, the process is complete - the total time is the network time, plus the walk time to the network. 
grid['total_time_net'] = grid['on_network_time'] + grid['walk_time_to_net']

### Calculate Direct Walking Time (not using road network), vs. network Time

In some cases, it won't be logical to use the network at all - walking to the network for a long time, only to drive for 2 mins and then walk back off the network again, will not be logical. To circumvent this, we also calculate what happens if you walk in a straight line towards your closest destination - ignoring roads. If this time is less than the network time, we use this instead. 

To do this, we take our dataframe of acceptable destinations, and snap the origin points _Directly_ to the destinations - then work out the corresponding elevations, and thus travel times. 

In [25]:
bundle = []
W = graph_node_gdf.copy()
W['node_ID'] = W['node_ID'].astype(str)
W = W.set_index('node_ID')

# Generate dictionary of locations by homogenously controlled polygon
locations_gdf = gpd.GeoDataFrame(acceptable_df, geometry = 'geometry', crs = {'init':'epsg:4326'})
locations_container = AggressiveSpatialIntersect(locations_gdf, merged_dists)

# for each polygon, snap the origins in that polygon to the acceptable destinations
for key in origin_container.keys():
    
    # select subset of all origin points - only the ones in this polygon
    origins = origin_container[key]
    origin_subset = grid.copy()
    origin_subset = origin_subset.loc[origins]
    
    # select subset of all locations - only the ones in this polygon
    locations = locations_gdf.loc[locations_container[key]]
    
    # control for no points of service in this polygon. Early end! No pandana snap. 
    if len(locations) < 1:
        origin_subset['NN'] = None
        origin_subset['NN_dist'] = None
        bundle.append(origin_subset)
    else:
        # Here, we find the closest hospital to each origin point as the crow-flies by using pandana snap
        origin_subset_snapped = gn.pandana_snap_points(origin_subset, 
                                locations, 
                                source_crs = 'epsg:4326', 
                                target_crs = 'epsg:32638', 
                                add_dist_to_node_col = True)
        bundle.append(origin_subset_snapped)

grid_gdf_adjusted = pd.concat(bundle)

grid = grid_gdf_adjusted

# Y is now a copy of the grid to save space
Y = grid.copy()

objs = []
if len(Y.loc[Y['NN'].isnull() == True]) > 0:
    Y2 = Y.loc[Y['NN'].isnull() == True]
    Y2['walkspeed_direct'] = 0
    Y2['walk_time_direct'] = 9999999
    Y2['NN_dist_direct'] = 9999999
    objs.append(Y2)

# add location elevations
location_elevs = add_elevation(locations_gdf, 'Longitude','Latitude', srtm_pth)

# match on to the grid the elevation of the nearest location to that point
Y1 = Y.loc[Y['NN'].isnull() == False]
Y1['NN'] = Y1['NN'].astype(int)
Y1 = Y1.set_index('NN')
Y1['dest_NN_elev'] = location_elevs['point_elev']

Y1 = Y1.reset_index()

# generate walktimes for each grid point directly to the nearest point of service
Y1 = generate_walktimes(Y1, start = 'point_elev', end = 'dest_NN_elev', dist = 'NN_dist', max_walkspeed = offroad_speed).reset_index()

# housekeeping - rename columns for easier identification
Y1 = Y1.rename({'walkspeed':'walkspeed_direct', 
                    'walk_time':'walk_time_direct',
                   'NN_dist':'NN_dist_direct'}, axis = 1)
objs.append(Y1)

grid = pd.concat(objs)

# take the MINIMUM travel time of a.) directly walking there, and b.) using the network
grid['PLOT_TIME_SECS'] = grid[['walk_time_direct','total_time_net']].min(axis = 1)

# convert to minutes
grid['PLOT_TIME_MINS'] = grid['PLOT_TIME_SECS'] / 60

cutting geometry 56 into 121 pieces
cutting geometry 59 into 47 pieces
cutting geometry 78 into 236 pieces
cutting geometry 79 into 16 pieces


### Generate Output Raster

At this point, all of the maths is done. We want to load our results onto a raster for visualization. We start by copying the input raster - the Worldpop grid - which made all of this possible. Then, we adjust the metadata to allow for another band of data to be added. This band will represent the travel time to the closest facility for that grid cell. Hence, the resultant raster will be dual-band - the first layer, the population, and the second layer, the accessibility of the people living in that gridcell.

In [28]:
# This is the raster we start from
rst_fn = os.path.join(pth,'pop18_resampled.tif')

# set output filename + filepath
out_fn = os.path.join(basepth,'output_layers','Round 3','%s.tif' % subset)

# Copy the metadata of the WorldPop raster
rst = rt.open(rst_fn, 'r')
meta = rst.meta.copy()
D_type = rt.float64

# By upping count to 2, we allow for a second data band
meta.update(compress='lzw', dtype = D_type, count = 2)

# open both the template file and the new file
with rt.open(out_fn, 'w', **meta) as out:
    with rt.open(rst_fn, 'r') as pop:
        
        # this is where we create a generator of geom, value pairs to use in rasterizing
        shapes = ((geom,value) for geom, value in zip(grid.geometry, grid.PLOT_TIME_MINS))
        
        # we copy our population layer
        population = pop.read(1).astype(D_type)
        cpy = population.copy()
        
        # to generate the travel times, we rasterize the geometry: value pairs in the Grid GeoDataFrame
        travel_times = features.rasterize(shapes=shapes, fill=0, out=cpy, transform=out.transform)

        # We then write these bands out, one after another
        out.write_band(1, population)
        out.write_band(2, travel_times)
        
# That's it - we are done for this scenario
print('**process complete**')

**process complete**


### Generate Zonal Stats

This function is taken from Ben Stewart's GOSTRocks library. The only difference is the setting of 'all touched' to False to prevent over-counting of the population. 

We will use this function to summarize the number of people, in any given polygon, that have access / lack access to a functioning, valid facility within X minutes. 

In [29]:
def zonalStats(inShp, inRaster, bandNum=1, mask_A = None, reProj = False, minVal = '', maxVal = '', verbose=False , rastType='N', unqVals=[]):
    import sys, os, inspect, logging, json
    import rasterio, affine

    import pandas as pd
    import geopandas as gpd
    import numpy as np

    from collections import Counter
    from shapely.geometry import box
    from affine import Affine
    from rasterio import features
    from rasterio.mask import mask
    from rasterio.features import rasterize
    from rasterio.warp import reproject, Resampling
    from osgeo import gdal
    
    ''' Run zonal statistics against an input shapefile
    
    INPUT VARIABLES
    inShp [string or geopandas object] - path to input shapefile
    inRaster [string or rasterio object] - path to input raster
    
    OPTIONAL
    bandNum [integer] - band in raster to analyze
    reProj [boolean] -  whether to reproject data to match, if not, raise an error
    minVal [number] - if defined, will only calculation statistics on values above this number
    verbose [boolean] - whether to be loud with responses
    rastType [string N or C] - N is numeric and C is categorical. Categorical returns counts of numbers
    unqVals [array of numbers] - used in categorical zonal statistics, tabulates all these numbers, will report 0 counts
    mask_A [numpy boolean mask] - mask the desired band using an identical shape boolean mask. Useful for doing conditional zonal stats
    
    RETURNS
    array of arrays, one for each feature in inShp
    '''   
    if isinstance(inShp, str):
        inVector = gpd.read_file(inShp) 
    else:
        inVector = inShp
    if isinstance(inRaster, str):
        curRaster = rasterio.open(inRaster, 'r+')
    else:
        curRaster = inRaster
        
    # If mask is not none, apply mask 
    if mask_A is not None:
        
        curRaster.write_mask(np.invert(mask_A))
    
    outputData=[]
    if inVector.crs != curRaster.crs:
        if reProj:
            inVector = inVector.to_crs(curRaster.crs)
        else:
            raise ValueError("Input CRS do not match")
    fCount = 0
    tCount = len(inVector['geometry'])
    #generate bounding box geometry for raster bbox
    b = curRaster.bounds
    rBox = box(b[0], b[1], b[2], b[3])
    for geometry in inVector['geometry']:
        #This test is used in case the geometry extends beyond the edge of the raster
        #   I think it is computationally heavy, but I don't know of an easier way to do it
        if not rBox.contains(geometry):
            geometry = geometry.intersection(rBox)            
        try:
            fCount = fCount + 1
            if fCount % 1000 == 0 and verbose:
                tPrint("Processing %s of %s" % (fCount, tCount) )
            # get pixel coordinates of the geometry's bounding box
            ul = curRaster.index(*geometry.bounds[0:2])
            lr = curRaster.index(*geometry.bounds[2:4])
            '''
            TODO: There is a problem with the indexing - if the shape falls outside the boundaries, it errors
                I want to change it to just grab what it can find, but my brain is wrecked and I cannot figure it out
            print(geometry.bounds)
            print(curRaster.shape)
            print(lr)
            print(ul)
            lr = (max(lr[0], 0), min(lr[1], curRaster.shape[1]))
            ul = (min(ul[0], curRaster.shape[0]), min(ul[1]))
            '''
            # read the subset of the data into a numpy array
            window = ((float(lr[0]), float(ul[0]+1)), (float(ul[1]), float(lr[1]+1)))
            
            if mask is not None:
                data = curRaster.read(bandNum, window=window, masked = True)
            else:
                data = curRaster.read(bandNum, window=window, masked = False)
            
            # create an affine transform for the subset data
            t = curRaster.transform
            shifted_affine = Affine(t.a, t.b, t.c+ul[1]*t.a, t.d, t.e, t.f+lr[0]*t.e)

            # rasterize the geometry
            mask = rasterize(
                [(geometry, 0)],
                out_shape=data.shape,
                transform=shifted_affine,
                fill=1,
                all_touched=False,
                dtype=np.uint8)

            # create a masked numpy array
            masked_data = np.ma.array(data=data, mask=mask.astype(bool))
            if rastType == 'N':                
                if minVal != '' or maxVal != '':
                    if minVal != '':
                        masked_data = np.ma.masked_where(masked_data < minVal, masked_data)
                    if maxVal != '':
                        masked_data = np.ma.masked_where(masked_data > maxVal, masked_data)                    
                    if masked_data.count() > 0:                        
                        results = [masked_data.sum(), masked_data.min(), masked_data.max(), masked_data.mean()]
                    else :
                        results = [-1, -1, -1, -1]                
                else:
                    results = [masked_data.sum(), masked_data.min(), masked_data.max(), masked_data.mean()]
            if rastType == 'C':
                if len(unqVals) > 0:                          
                    xx = dict(Counter(data.flatten()))
                    results = [xx.get(i, 0) for i in unqVals]                
                else:
                    results = np.unique(masked_data, return_counts=True)                    
            outputData.append(results)
        except Exception as e: 
            print(e)
            outputData.append([-1, -1, -1, -1])            
    return outputData   

# Additional Analyses / Visualization Aids

The main process is now finished. However, in order to generate the many output images and special analyses for the Yemen project, standalone sub-processes have been devised to generate outputs like: 
- District and national level zonal statistics
- 2016 v 2018 change in access detection
- Mash-ups with the vulnerability matrix data
We detail four of these examples below. Other can be found at the foot of the automated version of the file (these were used later). 

### Running Zonal Stats

Here, we make use of our zonal statistics function to generate summary values for an administrative boundary layer. 

The script is currently set up to accept either a national (adm-0) level shapefile, or a district (adm-2) level shapefile - though, with some cursory and superficial modification, it could be used to generate zonal stats for any given polygon.

In [30]:
# sometimes, we may want to run the script from top to bottom without doing this bit. Thus, we have a control variable, 
# zonal_stats, which if set to 0 will disable this chunk. Ensure zonal_stats != 0 to continue. 
if zonal_stats == 0:
    pass

else:
    # These are the two shapefiles we have routinely used. We generate outputs for both. 
    for resolution in ['national','district']:
        
        # We pick up our raster from the write location (Don't change the output names!)
        out_fn = os.path.join(r'C:\Users\charl\Documents\GOST\Yemen\output_layers','Round 3',
                              '%s.tif' % subset)
        
        # Set the util path - where we can load the admin boundary polygon from
        utils = r'C:\Users\charl\Documents\GOST\Yemen\util_files'

        # Here we load the national-level shapefile - the Yemen bound
        yemen_shp_name = os.path.join(utils, r'Yemen_bound.shp')
        yemen_shp = gpd.read_file(yemen_shp_name)
        
        # Reproject to WGS84 if necessary
        if yemen_shp.crs != {'init': 'epsg:4326'}:
            yemen_shp = yemen_shp.to_crs({'init': 'epsg:4326'})
        
        # Pick up the district shapefile, and reproject if necessary
        district_shp_name = os.path.join(r'C:\Users\charl\Documents\GOST\Yemen\VulnerabilityMatrix', r'VM.shp')
        district_shp = gpd.read_file(district_shp_name)
        
        if district_shp.crs != {'init': 'epsg:4326'}:
            district_shp = district_shp.to_crs({'init': 'epsg:4326'})
        
        # Here we read in the output raster we just created
        inraster = out_fn
        ras = rt.open(inraster, mode = 'r+')
        
        # pop is the population band
        pop = ras.read(1)
        
        # tt_matrix is the travel time band
        tt_matrix = ras.read(2)
        
        # set the target shape
        if resolution == 'national':
            target_shp = yemen_shp
        elif resolution == 'district':
            target_shp = district_shp

        # Add on the total population of the district to each district shape
        mask_pop = np.ma.masked_where(pop > (200000), pop).mask

        base_pop = zonalStats(target_shp,     # target shp
                                inraster,     # our output raster
                                bandNum = 1,  # pick band 1 - the population band
                                mask_A = mask_pop, # use mask_pop, which removes values greater than 200,000 (i.e. errors)
                                reProj = False, # do not reproject
                                minVal = 0,
                                maxVal = np.inf, 
                                verbose = True, 
                                rastType='N')  # as opposed to categorical
        
        # the zonalStats function returns 4 outputs - sum, min, max and mean. We only want sum
        cols = ['total_pop','min','max','mean']
        
        temp_df = pd.DataFrame(base_pop, columns = cols)
        
        # we match the sum back on to our original target_shp file
        target_shp['total_pop'] = temp_df['total_pop']
        target_shp['total_pop'].loc[target_shp['total_pop'] == -1] = 0

        # Having added the base population, we now calculate the population within a range 
        # of time thresholds from the destination set. 
        # we do this for four time thresholds. Note the change in the mask argument:
        for time_thresh in [30,60,120, 240]:
            
            # this is our new, special, mask - we mask all values ABOVE the threshold
            mask_obj = np.ma.masked_where(tt_matrix > (time_thresh), tt_matrix).mask
            
            # we pass this to our same zonal stats function. We are only counting population values below the threshold now
            raw = zonalStats(target_shp, 
                                inraster, 
                                bandNum = 1,
                                mask_A = mask_obj,
                                reProj = False, 
                                minVal = 0,
                                maxVal = np.inf, 
                                verbose = True, 
                                rastType='N')

            # create  temp_df of the results
            cols = ['pop_%s' % time_thresh,'min','max','mean']
            temp_df = pd.DataFrame(raw, columns = cols)
            
            # Add in this new populaiton count to the file
            target_shp['pop_%s' % time_thresh] = temp_df['pop_%s' % time_thresh]
            target_shp['pop_%s' % time_thresh].loc[target_shp['pop_%s' % time_thresh] == -1] = 0
            
            # note the percentage of the total population that has access within this thresh
            target_shp['frac_%s' % time_thresh] = (target_shp['pop_%s' % time_thresh]) / (target_shp['total_pop']).fillna(0)
            target_shp['frac_%s' % time_thresh].replace([np.inf, -np.inf], 0)
            target_shp['frac_%s' % time_thresh] = target_shp['frac_%s' % time_thresh].fillna(0)

        # Save to file. The only difference is that we save a summary .csv instead of a .shp for national - as we never want to 
        # visualize the national-level results (it would be one single block of color)

        if resolution == 'national':
            print('saving national')
            outter = target_shp[['total_pop','pop_30','frac_30','pop_60','frac_60','pop_120','frac_120','pop_240','frac_240']]
            outter.to_csv(os.path.join(basepth, 'output_layers','Round 3','%s_zonal_%s.csv'% (subset, resolution)))
        
        else:
            print('saving district')
            target_shp['abs_pop_iso'] = target_shp['total_pop'] - target_shp['pop_30']
            target_shp.to_file(os.path.join(basepth, 'output_layers','Round 3','%s_zonal_%s.shp' % (subset, resolution)), driver = 'ESRI Shapefile')

C:\Users\charl\Anaconda3\envs\Cfox2\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


saving national
saving district


### Generate Change Raster

This analysis assumes that the user has already run a scenario for 2016, and a scenario for 2018 - with all other settings held entirely constant. 

We do some basic raster maths on the travel time band of two output rasters to generate a delta / change raster. 

In [31]:
# set to 'off' to continue
test_mode = 'on'

if test_mode == 'on':
    pass

else:
    # name the output file
    subset = r'PHCs_2018_conflict_delta'
    
    # this is the first raster 
    pre_raster = os.path.join(basepth, 'output_layers','Round 3','driving_24th_HERAMS_PHC_NoConflict.tif')
    
    # this is the second raster
    post_raster = os.path.join(basepth, 'output_layers','Round 3','driving_24th_HERAMS_PHCs_ConflictAdj.tif')
    
    # define the output / delta / change raster filename and location
    out_fn = os.path.join(basepth,'output_layers','Round 3','%s.tif' % subset)
    
    # open the first raster, read in band 2
    pre = rasterio.open(pre_raster, 'r')
    arr_pre = pre.read(2)
    
    # repeat for second raster
    post = rasterio.open(post_raster, 'r')
    arr_post = post.read(2)
    
    # subtract post from pre
    delta = arr_pre - arr_post

    # Update metadata from template raster
    rst_fn = os.path.join(pth,'pop18_resampled.tif')
    rst = rasterio.open(rst_fn, 'r')
    meta = rst.meta.copy()
    D_type = rasterio.float64
    meta.update(compress='lzw', dtype = D_type, count = 3)
    
    # write out change raster
    with rasterio.open(out_fn, 'w', **meta) as out:
        with rasterio.open(rst_fn, 'r') as pop:

            # keep the original population later
            population = pop.read(1).astype(D_type)
            
            # the only surprise here is band 3 - the number of people x the minutes of change. 
            # Useful for zonal stats to identify the areas which changed the most for the better or worse.
            out.write_band(1, population)
            out.write_band(2, delta)
            out.write_band(3, delta * population)

### Relationship Graphs: Vulnerability Component Scores, Accessibility

Here, we generate some line plots to try to identify any simple / obvious relationships between Vulnerability Matrix factor scores and accessibility. 

In [32]:
# adjust test_mode variable to != on to continue
if test_mode == 'on':
    pass
else:
    import seaborn as sns
    import matplotlib.pyplot as plt
    
    # Here, we need to import a prepared target_shp file. This has the VM details in it + access stats from the zonal stats step.
    # this is important - it must have both VM data AND access data from zonal stats in the same file. 
    # the path below is the path to the raw VM file - which should then be used in the zonal stats step before running this block.
    target_shp = os.path.join(r'C:\Users\charl\Documents\GOST\Yemen\VulnerabilityMatrix', r'VM.shp')
    
    # rename columns for easier access
    target_shp2 = target_shp.rename({
        'Overall Vu':'Overall Vulnerability Level',
        'Health Sys':'Health System Capacity Score',
        'Hazards':'Hazard Score',
        'Impact on':'Impact on Exposed Population Score',
        'Food Secur':'Food Security Score',
        'Morbidity':'Morbidity Score',
        'Nutrition':'Nutrition Score',
        'WASH':'WASH Score',
        'Social Det':'Social Determinants and Health Outcomes Score',
        'total_pop':'Total Population',
                                   }, axis = 1)
    
    # pick out the factors / scores we want to generate graphs for
    factors = ['Overall Vulnerability Level','Health System Capacity Score',
              'Hazard Score','Impact on Exposed Population Score','Food Security Score',
              'WASH Score','Social Determinants and Health Outcomes Score']
    
    # define our time thresholds of interest
    fracs = {'frac_30':'30 minutes',
            'frac_60': '1 hour',
            'frac_120': '2 hours',
            'frac_240': '4 hours'}
    
    # now, for each factor
    for groupa in factors:
        
        # we group by each factor in turn. These factors have values between 1 and 7, so we have c. 8 groups
        subg = target_shp2[[groupa,'Total Population','pop_30','pop_60','pop_120','pop_240']].groupby(groupa).sum()
        
        # work out the fraction which has access in each district
        for i in [30, 60, 120, 240]:
            subg['frac_%s' % i] = subg['pop_%s' % i] / subg['Total Population']
            
        #  We plot out the average fraction that has access for each value present in the VM factor
        plotter = subg[['frac_30','frac_60','frac_120','frac_240']]
        plt.clf()
        plt.figure(figsize=(8,4))
        
        # title says it all
        title = 'Fraction of Population with access to a HeRAMS hospital for each value \nof the WHO %s' % (groupa)
        
        # set color palette
        pal = sns.cubehelix_palette(4, start=2.7, rot=.1, light = .7, dark=.1)
        ax = sns.lineplot(data = plotter, 
                          palette = pal, 
                          dashes = False).set_title(title)
        
        # add a legend and save the plot down
        plt.legend(loc='center right', bbox_to_anchor=(1.2, 0.5), ncol=1)
        plt.savefig(os.path.join(r'C:\Users\charl\Documents\GOST\Yemen\output_layers\VM_graphs','pop_chart_%s' % groupa), bbox_inches='tight')

### Scatter plots: Vulnerability Matrix

The previous analysis sees us generate plots which are summaries for each VM score for a given factor. Although interesting, it is also useful to see the distribution of districts within each VM factor band - so we can see if there is tight grouping, or not. We create a scatter plot to do that here. The code is extremely similar to the previous box.


In [33]:
if test_mode == 'on':
    pass
else:
    
    import seaborn as sns
    import matplotlib.pyplot as plt
    
    # again, make sure you have a correctly set up 'target_shp' file
    target_shp2 = target_shp.rename({
        'Overall Vu':'Overall Vulnerability Level',
        'Health Sys':'Health System Capacity Score',
        'Hazards':'Hazard Score',
        'Impact on':'Impact on Exposed Population Score',
        'Food Secur':'Food Security Score',
        'Morbidity':'Morbidity Score',
        'Nutrition':'Nutrition Score',
        'WASH':'WASH Score',
        'Social Det':'Social Determinants and Health Outcomes Score',
        'total_pop':'Total Population',
                                   }, axis = 1)
    
    p = 7

    factors = ['Overall Vulnerability Level','Health System Capacity Score',
              'Hazard Score','Impact on Exposed Population Score','Food Security Score',
              'WASH Score','Social Determinants and Health Outcomes Score']

    fracs = {'frac_30':'30 minutes',
            'frac_60': '1 hour',
            'frac_120': '2 hours',
            'frac_240': '4 hours'}
    
    # we generate scatter plots for each accessibility threshold
    for frac in ['frac_30','frac_60','frac_120','frac_240']:
        
        # and for each Vulnerability matrix factor
        for groupa in factors:
            plt.clf()
            plt.figure(figsize=(8,4))
            title = 'Fraction of district population living within %s of nearest HeRAMS hospital, \nbreakdown by WHO %s' % (fracs[frac], groupa)
            
            # this time, we are not running a groupby function - so we plot all values
            d = target_shp2[[groupa,frac]]
            
            # set color palette
            pal = sns.cubehelix_palette(7, rot=-.5, light = .8, dark=.2)
            
            # plot on the axes
            ax = sns.swarmplot(y = frac, x = groupa, data=d, palette=pal).set_title(title)
            
            # generate some boxplots to go on top to describe the data
            ax = sns.boxplot(y = frac, x = groupa, data=d, whis=np.inf, color = "1", linewidth = 0.7, dodge = True)
            
            # save down
            plt.savefig(os.path.join(r'C:\Users\charl\Documents\GOST\Yemen\output_layers\VM_graphs','Boxplot_%s_%s.png'% (groupa, frac)), bbox_inches='tight')